In [1]:
import requests
from tqdm import tqdm
from os.path import join as oj
import tables, numpy
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi
from skimage import data
import pickle as pkl
from skimage.util import img_as_float
from sklearn import metrics
import h5py
from copy import deepcopy
from skimage.filters import gabor_kernel
import gabor_feats
from sklearn.linear_model import RidgeCV
import seaborn as sns
import numpy.linalg as npl
out_dir = '/scratch/users/vision/data/gallant/vim_2_crcns'
from run import *

# download data

In [48]:
def download(datafile, username, password, out_dir):
    '''
    Params
    ------
    datafile
    '''
    
    URL = 'https://portal.nersc.gov/project/crcns/download/index.php'
    login_data = dict(
        username=username,
        password=password,
        fn=datafile,
        submit='Login' 
    )

    with requests.Session() as s:
        local_filename = oj(out_dir, login_data['fn'].split('/')[-1])
        print(local_filename)
        r = s.post(URL, data=login_data, stream=True)
        with open(local_filename, 'wb') as f:
            for chunk in tqdm(r.iter_content(chunk_size=1024)):
                if chunk:
                    f.write(chunk)
                    
uname = 'csinva'
pwd = 'password'
dset = 'vim-2'
fnames = ['Stimuli.tar.gz', 'VoxelResponses_subject1.tar.gz', 'anatomy.zip', 'checksums.md5', 'filelist.txt', 'docs']
for fname in fnames:
    fname = oj(dset, fname)
#     download(fname, uname, pwd, out_dir)

In [49]:
ls /scratch/users/vision/data/gallant/vim_2_crcns

anatomy.zip    docs          Stimuli.mat     VoxelResponses_subject1.mat
checksums.md5  filelist.txt  Stimuli.tar.gz  VoxelResponses_subject1.tar.gz


In [23]:
!du -sh /scratch/users/vision/data/gallant/vim_2_crcns
# next extract the tars
# next unzip the zips

6.9G	/scratch/users/vision/data/gallant/vim_2_crcns


In [27]:
!ls /scratch/users/vision/data/gallant/vim_2_crcns/*.gz |xargs -n1 tar -xzf # extract the tar files

# view responses

In [ ]:
f = tables.open_file(oj(out_dir, 'VoxelResponses_subject1.mat'))
# f.listNodes # Show all variables available
data = f.get_node('/rt')[:] # training responses: 7200 (timepoints) x 73728
# plt.imshow(np.isnan(data))
roi = f.get_node('/roi/v1lh')[:].flatten() # structure containing volume matrices (64x64x18) with indices corresponding to each roi in each hemisphere
v1lh_idx = numpy.nonzero(roi==1)[0]
v1lh_resp = data[v1lh_idx]

In [ ]:
f2 = tables.open_file(oj(out_dir, 'Stimuli.mat'))
im = f2.get_node('/st')[100].transpose()
plt.imshow(im)

In [134]:
str(xs[0]).split(' ')[0]

['/roi/FFAlh', '(EArray(18,', '64,', '64),', 'zlib(3))', "''"]

In [168]:
f = tables.open_file(oj(out_dir, 'VoxelResponses_subject1.mat'))
xs = []
nums = []
for x in f.get_node('/roi'):
    xs.append(x)
    nums.append(np.array(f.get_node(x)).nonzero()[0].sum())
# sns.barplot(x=x, y=nums)
print([str(x) for x in xs])

["/roi/FFAlh (EArray(18, 64, 64), zlib(3)) ''", "/roi/FFArh (EArray(18, 64, 64), zlib(3)) ''", "/roi/IPlh (EArray(18, 64, 64), zlib(3)) ''", "/roi/IPrh (EArray(18, 64, 64), zlib(3)) ''", "/roi/MTlh (EArray(18, 64, 64), zlib(3)) ''", "/roi/MTplh (EArray(18, 64, 64), zlib(3)) ''", "/roi/MTprh (EArray(18, 64, 64), zlib(3)) ''", "/roi/MTrh (EArray(18, 64, 64), zlib(3)) ''", "/roi/OBJlh (EArray(18, 64, 64), zlib(3)) ''", "/roi/OBJrh (EArray(18, 64, 64), zlib(3)) ''", "/roi/PPAlh (EArray(18, 64, 64), zlib(3)) ''", "/roi/PPArh (EArray(18, 64, 64), zlib(3)) ''", "/roi/RSCrh (EArray(18, 64, 64), zlib(3)) ''", "/roi/STSrh (EArray(18, 64, 64), zlib(3)) ''", "/roi/VOlh (EArray(18, 64, 64), zlib(3)) ''", "/roi/VOrh (EArray(18, 64, 64), zlib(3)) ''", "/roi/latocclh (EArray(18, 64, 64), zlib(3)) ''", "/roi/latoccrh (EArray(18, 64, 64), zlib(3)) ''", "/roi/v1lh (EArray(18, 64, 64), zlib(3)) ''", "/roi/v1rh (EArray(18, 64, 64), zlib(3)) ''", "/roi/v2lh (EArray(18, 64, 64), zlib(3)) ''", "/roi/v2rh (EAr

In [15]:
# calculate standard deviations
f = tables.open_file(oj(out_dir, 'VoxelResponses_subject1.mat'))
rva = np.array(f.get_node('/rva')[:]) # 73728 (voxels) x 10 (trials) x 540 (timepoints)
sigmas = np.nanmean(np.nanstd(rva, axis=1), axis=-1)
out_name = oj(out_dir, f'out_rva_sigmas.h5')
save_h5(sigmas, out_name)

/accounts/projects/vision/.local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/accounts/projects/vision/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.


# view images

In [160]:
SAMPLING_FREQ = 15
DOWNSAMPLE = 2
N_TRAIN = 7200
N_TEST = 540
OFFSET = SAMPLING_FREQ // 2
NUM_FEATS = 1280

In [ ]:
# find the relevant stimuli
for dset, N in zip(['sv'], [N_TEST]): # 'st', 'sv'
    f2 = tables.open_file(oj(out_dir, 'Stimuli.mat'))
    ims = np.zeros((N, 128 // DOWNSAMPLE, 128 // DOWNSAMPLE)).astype(np.int)
    for i in tqdm(range(N)):
        ims[i] = deepcopy(f2.get_node(f'/{dset}')[OFFSET + i * SAMPLING_FREQ].transpose())[::DOWNSAMPLE, ::DOWNSAMPLE].mean(axis=-1)

    out_name = oj(out_dir, f'out_{dset}.h5')
    save_h5(ims, out_name)

In [ ]:
# convert stimuli to feature vectors
for dset, N in zip(['sv'], [N_TEST]): # 'st', 'sv'
    f = h5py.File(oj(out_dir, f'out_{dset}.h5'), 'r')
    feats = np.zeros((N, NUM_FEATS))
    for i in tqdm(range(N)):
        feats[i] = gabor_feats.all_feats(deepcopy(f['data'][i]))

    out_name = oj(out_dir, f'out_{dset}_feats.h5')
    save_h5(feats, out_name)

In [10]:
# decompose the training data
'''
suffix = '' # _feats
X = np.array(h5py.File(oj(out_dir, f'out_st{suffix}.h5'), 'r')['data'])
X = X.reshape(X.shape[0], -1)
U, s, Vh = npl.svd(X)
save_pkl((U, s, Vh), oj(out_dir, f'decomp{suffix}.pkl'))
'''

"\nsuffix = '' # _feats\nX = np.array(h5py.File(oj(out_dir, f'out_st{suffix}.h5'), 'r')['data'])\nX = X.reshape(X.shape[0], -1)\nU, s, Vh = npl.svd(X)\nsave_pkl((U, s, Vh), oj(out_dir, f'decomp{suffix}.pkl'))\n"

In [10]:
# fit linear models
suffix = '_feats' # _feats, '' for pixels
rois = ['v1lh', 'v1rh', 'v2lh', 'v2rh', 'v4lh', 'v4rh']
NUM = 10
save_dir = '/scratch/users/vision/data/gallant/vim_2_crcns/feats1'
feats_name = oj(out_dir, f'out_st{suffix}.h5')
feats_test_name = oj(out_dir, f'out_sv{suffix}.h5')
resps_name = oj(out_dir, 'VoxelResponses_subject1.mat')


print('loading data...')
X = np.array(h5py.File(feats_name, 'r')['data'])
X = X.reshape(X.shape[0], -1)
Y = np.array(tables.open_file(resps_name).get_node('/rt')[:]) # training responses: 73728 (voxels) x 7200 (timepoints)
X_test = np.array(h5py.File(feats_test_name, 'r')['data'])
X_test = X_test.reshape(X_test.shape[0], -1)
Y_test = np.array(tables.open_file(resps_name).get_node('/rv')[:]) # training responses: 73728 (voxels) x 7200 (timepoints)
sigmas = load_h5(oj(out_dir, f'out_rva_sigmas.h5'))
(U, s, Vh) = pkl.load(open(oj(out_dir, f'decomp{suffix}.pkl'), 'rb'))

loading data...


In [11]:
np.isnan(U).sum()

0

In [12]:
np.max(U)

1.0

In [13]:
np.min(U)

-0.23158069795429415

In [ ]:
# actually run
os.makedirs(save_dir, exist_ok=True)
f = tables.open_file(oj(out_dir, 'VoxelResponses_subject1.mat'), 'r')
for roi in rois:
    roi_idxs = f.get_node(f'/roi/{roi}')[:].flatten().nonzero()[0] # structure containing volume matrices (64x64x18) with indices corresponding to each roi in each hemisphere
#         print(roi, roi_idxs.size)
    roi_idxs = roi_idxs[:NUM]
    results = {}

    for i in tqdm(roi_idxs):
        y = Y[i]
        y_test = Y_test[i]
        w = U.T @ y

        sigma = sigmas[i]
        var = sigma**2

        n = np.sum(~np.isnan(y))
        num_test = np.sum(~np.isnan(y_test))
        d = X.shape[1]
        d_n_min = min(n, d)
        if n == y.size and num_test == y_test.size: # ignore voxels w/ missing vals
            print('w has nan', np.isnan(w).sum())
            m = RidgeCV(alphas=[6, 10, 25, 50, 100])
            m.fit(X, y)
            preds = m.predict(X_test)
            mse = metrics.mean_squared_error(y_test, preds)
            r2 = metrics.r2_score(y_test, preds)
            corr = np.corrcoef(y_test, preds)[0, 1]
            # print('w', w, npl.norm(w))
            term1 = 0.5 * (npl.norm(y) ** 2 - npl.norm(w) ** 2) / var
            term2 = 0.5 * np.sum([np.log(1 + w[i]**2 / var) for i in range(d_n_min)])
            complexity1 = term1 + term2
            print('term1', term1, 'term2', term2) #, 'alpha', m.alpha_)

            idxs = np.abs(w) > sigma
            term3 = 0.5 * np.sum([np.log(1 + w[i]**2 / var) for i in np.arange(n)[idxs]])
            term4 = 0.5 * np.sum([w[i]**2 / var for i in np.arange(n)[~idxs]])
            complexity2 = term1 + term3 + term4

            results = {
                'roi': roi,
                'model': m,
                'term1': term1,
                'term2': term2,
                'term3': term3,
                'term4': term4,
                'complexity1': complexity1,
                'complexity2': complexity2,
                'num_train': n,
                'num_test': num_test,
                'd': d,
                'mse': mse,                
                'r2': r2,
                'corr': corr
            }
            pkl.dump(results, open(oj(save_dir, f'ridge_{i}.pkl'), 'wb'))

In [3]:
U.shape

(7200, 7200)

In [ ]:
for i in range(U.shape[0]):
    print(np.linalg.norm(U[i]))

In [7]:
var

0.02085533674759077

In [9]:
print('w', w, npl.norm(w))
term1 = 0.5 * (npl.norm(y) ** 2 - npl.norm(w) ** 2) / var
term2 = 0.5 * np.sum([np.log(1 + w[i]**2 / var) for i in range(d_n_min)])
complexity1 = term1 + term2

idxs = np.abs(w) > sigma
term3 = 0.5 * np.sum([np.log(1 + w[i]**2 / var) for i in np.arange(n)[idxs]])
term4 = 0.5 * np.sum([w[i]**2 / var for i in np.arange(n)[~idxs]])
print('term1', term1, 'term2', term2, 'term3', term3, 'term4', term4) #, 'alpha', m.alpha_)

complexity2 = term1 + term3 + term4

w [ 0.2929799   1.37100595 -1.57911986 ... -1.03751737  1.30191682
  0.27016271] 84.83333079080593
term1 -0.010347162671320824 term2 6051.0455633117635 term3 10484.672144762035 term4 133.0504771636347


In [ ]:
np